<a href="https://colab.research.google.com/github/Sarthak-125/PINN_2/blob/main/Burger's_Equation_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [2]:
import numpy as np
import tensorflow as  tf
import random

BC u(t,1) = u(t,-1) = 0

Collocation point creation



In [3]:
n_c = 5000  #collocation points

def Rand(start, end, num): 
    res = [] 
  
    for j in range(num): 
        res.append(random.uniform(start, end)) 
  
    return res

x_c = Rand(-1, 1, n_c)
t_c = Rand(0, 1, n_c)
x_c = tf.cast(x_c ,dtype='float32')
t_c = tf.cast(t_c ,dtype='float32')
input_c = tf.stack((x_c, t_c), axis=1)
x_bc_u = np.ones((n_c,), dtype='float32')
x_bc_b = - x_bc_u
x_bc_u = tf.cast(x_bc_u, dtype='float32')
x_bc_b = tf.cast(x_bc_b, dtype='float32')
input_bcu = tf.stack((x_bc_u, t_c), axis = 1)
input_bcb = tf.stack((x_bc_b, t_c), axis = 1)
input_bc = tf.concat((input_bcu, input_bcb), axis=0)
input_f = tf.concat((input_c, input_bc), axis=0)
x_f = input_f[:,0:1]
t_f = input_f[:,1:2]
y_bc = 0
out_bc = (y_bc* np.ones((2*n_c,1), dtype='float32'))
out_bc = tf.cast(out_bc, dtype='float32')

In [15]:
print(x_f)

tf.Tensor(
[[ 0.9332978 ]
 [ 0.4441381 ]
 [ 0.77020186]
 ...
 [-1.        ]
 [-1.        ]
 [-1.        ]], shape=(15000, 1), dtype=float32)


Architecture ANN

In [5]:
x_in = tf.keras.layers.Input(shape=(1), name="Position")
t_in = tf.keras.layers.Input(shape=(1), name="Time")
IN = tf.keras.layers.Concatenate(axis=-1, name="Input")([x_in, t_in])
dense1=(tf.keras.layers.Dense(50,activation='tanh'))(IN)
dense2=(tf.keras.layers.Dense(50,activation='tanh'))(dense1)
dense3=(tf.keras.layers.Dense(50,activation='tanh'))(dense2)
dense4=(tf.keras.layers.Dense(50,activation='tanh'))(dense3)
dense5=(tf.keras.layers.Dense(50,activation='tanh'))(dense4)
output=(tf.keras.layers.Dense(1,activation='tanh'))(dense5)
pinn1 = tf.keras.Model(inputs=[x_in,t_in], outputs=output, name="pinn1")

Custom Loss function

In [6]:
def custom_loss_1(arch, x, t):
  def loss(u_train, u_pred):
    with tf.GradientTape(persistent=True) as t1:
      t1.watch(x)
      t1.watch(t)
      with tf.GradientTape() as t2:
        t2.watch(x)
        u = arch([x, t])
        u_x = t2.gradient(u, x)
      u_t = t1.gradient(u, t)
      u_xx = t1.gradient(u_x, x)
    u_pred = tf.cast(u_pred, dtype='float32')
    u_train = tf.cast(u_train, dtype='float32')      
    f = u_t + u*u_x - (0.01/np.pi)*u_xx
    loss_res = tf.reduce_mean(tf.square(f))
    loss_2 = tf.reduce_mean(tf.square(u_pred-u_train))
    return loss_res + loss_2
  return loss


loss_fn = custom_loss_1(pinn1, x_f, t_f)
pinn1.compile(optimizer='adam', loss=loss_fn, metrics=['mse'])
pinn1.fit([input_bc[:,0:1], input_bc[:,1:2]], out_bc, batch_size=32, epochs=4)

Epoch 1/4
313/313 [==============================] - 74s 233ms/step - loss: 0.0027 - mse: 0.0015
Epoch 2/4
313/313 [==============================] - 74s 235ms/step - loss: 1.3197e-07 - mse: 3.0589e-09
Epoch 3/4
313/313 [==============================] - 74s 237ms/step - loss: 4.5503e-08 - mse: 6.5970e-10
Epoch 4/4
313/313 [==============================] - 74s 237ms/step - loss: 3.5809e-06 - mse: 2.7728e-06


In [31]:
x_test = x_f[random.randint(0,3*n_c)]
t_test = t_f[random.randint(0,3*n_c)]
u = pinn1([x_test, t_test])
tf.print("u at x=", x_test, "and at time t = ", t_test, "is =" , u)


u at x= [0.967259049] and at time t =  [0.550383151] is = [[-0.00161595282]]
